# colab setting, read file and import module

In [1]:
#!pip install google.colab #如未安裝取消註解後執行
import os
from google.colab import drive
drive.mount('/content/my_drive')

Mounted at /content/my_drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
file_path = '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/591_cleaned_data.csv'
df = pd.read_csv(file_path)

In [ ]:
df.info(max_cols=149)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 98 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           51073 non-null  int64  
 1   total_price          51073 non-null  int64  
 2   building_age         48588 non-null  float64
 3   total_area_ping      51073 non-null  float64
 4   address              51073 non-null  object 
 5   property_type        51073 non-null  object 
 6   usage_category       51073 non-null  object 
 7   latitude             51073 non-null  float64
 8   longitude            51073 non-null  float64
 9   community_name       39271 non-null  object 
 10  has_parking_space    51073 non-null  int64  
 11  total_floors         51073 non-null  float64
 12  transaction_floors   51073 non-null  float64
 13  target_floors        51073 non-null  float64
 14  bedrooms             49395 non-null  float64
 15  living_rooms         48002 non-null 

# 離群值濾除
* 邏輯上應該是要先濾除outlier再補值(至少補mean要這樣做)

In [3]:
df_clean = df.copy()

In [ ]:
df_clean['property_type'].value_counts()

property_type
電梯大樓    34145
公寓      12955
透天厝      2185
別墅       1788
Name: count, dtype: int64

In [ ]:
df_clean[df_clean['property_type'] == '透天厝']['new_per_ping'].describe()

count    2.185000e+03
mean     7.787527e+05
std      6.474876e+05
min      6.707669e+04
25%      3.961776e+05
50%      5.816267e+05
75%      9.520970e+05
max      1.014003e+07
Name: new_per_ping, dtype: float64

In [4]:
df_clean['property_type'].replace({'別墅': '透天厝'}, inplace=True)

In [ ]:
df_clean['property_type'].value_counts()

property_type
電梯大樓    34145
公寓      12955
透天厝      3973
Name: count, dtype: int64

In [ ]:
df_clean['new_per_ping'].describe()

count    5.107300e+04
mean     6.642425e+05
std      3.360482e+05
min      6.707669e+04
25%      4.444197e+05
50%      5.954128e+05
75%      8.062235e+05
max      1.014003e+07
Name: new_per_ping, dtype: float64

In [5]:
df_clean['property_type'].isnull().sum()

0

## 整體的total_area_ping
* 先看上面處理的如何再決定這邊

In [6]:
pings = df_clean['total_area_ping']

In [7]:
df_clean['total_area_ping'].isnull().sum()

0

In [ ]:
pings.describe()

count    51073.000000
mean        47.951388
std         33.557688
min          3.790000
25%         28.580000
50%         39.270000
75%         57.100000
max       1647.900000
Name: total_area_ping, dtype: float64

In [ ]:
df['Unnamed: 0']

0            0
1            1
2            3
3            4
4            5
         ...  
51068    55425
51069    55426
51070    55427
51071    55428
51072    55429
Name: Unnamed: 0, Length: 51073, dtype: int64

In [8]:
df['elevator_available'].isnull().sum()

0

In [ ]:
df_clean.info(max_cols=100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 98 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           51073 non-null  int64  
 1   total_price          51073 non-null  int64  
 2   building_age         48588 non-null  float64
 3   total_area_ping      51073 non-null  float64
 4   address              51073 non-null  object 
 5   property_type        51073 non-null  object 
 6   usage_category       51073 non-null  object 
 7   latitude             51073 non-null  float64
 8   longitude            51073 non-null  float64
 9   community_name       39271 non-null  object 
 10  has_parking_space    51073 non-null  int64  
 11  total_floors         51073 non-null  float64
 12  transaction_floors   51073 non-null  float64
 13  target_floors        51073 non-null  float64
 14  bedrooms             49395 non-null  float64
 15  living_rooms         48002 non-null 

# 是否有電梯的補值
* property_type 有電梯就是有電梯先補這樣
* 比較麻煩的是套房、店面就看不出有沒有電梯了

In [9]:
after_new_per_ping_outlier_df = df_clean.copy()

In [ ]:
after_new_per_ping_outlier_df['elevator_available'].value_counts()

elevator_available
0    34145
1    16928
Name: count, dtype: int64

In [10]:
# 轉成1代表有0代表沒有電梯的格式
after_new_per_ping_outlier_df['elevator_available'] = np.where(after_new_per_ping_outlier_df['elevator_available'].isin([1]), 0, 1)

In [ ]:
after_new_per_ping_outlier_df['elevator_available'].value_counts()

elevator_available
1    34145
0    16928
Name: count, dtype: int64

In [11]:
after_new_per_ping_outlier_df[['property_type', 'elevator_available']].value_counts()

property_type  elevator_available
電梯大樓           1                     34145
公寓             0                     12955
透天厝            0                      3973
Name: count, dtype: int64

# community_name有就給1沒有就0

In [12]:
after_new_per_ping_outlier_df['community_name'].value_counts()

community_name
宏盛水悅             189
宏盛海都 2期          138
太陽帝國             116
新外灘6-立信帝國花園廣場    116
台北晶麒             107
                ... 
麟光美寓               1
榮星超值2樓美寓           1
國揚林園別墅             1
忠誠路獨棟3房            1
成泰路一段7巷            1
Name: count, Length: 9000, dtype: int64

In [13]:
after_new_per_ping_outlier_df['community_name'].fillna(value=0, inplace=True)
after_new_per_ping_outlier_df['community_name'].isnull().sum()
after_new_per_ping_outlier_df['community_name'] = np.where(after_new_per_ping_outlier_df['community_name'].isin([0]), 0, 1)
after_new_per_ping_outlier_df['community_name'].unique()

array([0, 1])

# 看一下比鄰

In [14]:
near_fac_df = after_new_per_ping_outlier_df.copy() # 目前的主資料表

In [ ]:
############################## 存檔 ##########################################################################################################
# near_fac_df.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/EDA_main_data_version2_0627.csv', encoding='utf-8', index=True)

In [ ]:
near_fac_df.info(max_cols=154)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 98 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           51073 non-null  int64  
 1   total_price          51073 non-null  int64  
 2   building_age         48588 non-null  float64
 3   total_area_ping      51073 non-null  float64
 4   address              51073 non-null  object 
 5   property_type        51073 non-null  object 
 6   usage_category       51073 non-null  object 
 7   latitude             51073 non-null  float64
 8   longitude            51073 non-null  float64
 9   community_name       51073 non-null  int64  
 10  has_parking_space    51073 non-null  int64  
 11  total_floors         51073 non-null  float64
 12  transaction_floors   51073 non-null  float64
 13  target_floors        51073 non-null  float64
 14  bedrooms             49395 non-null  float64
 15  living_rooms         48002 non-null 

In [15]:
near_fac_df_2 = after_new_per_ping_outlier_df.iloc[:,20:95]

In [ ]:
near_fac_df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 75 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   公共活動空間_under_250_m  51073 non-null  float64
 1   公共活動空間_under_500_m  51073 non-null  float64
 2   公共活動空間_under_750_m  51073 non-null  float64
 3   交流道出入口_under_250_m  51073 non-null  float64
 4   交流道出入口_under_500_m  51073 non-null  float64
 5   交流道出入口_under_750_m  51073 non-null  float64
 6   捷運出口_under_250_m    51073 non-null  float64
 7   捷運出口_under_500_m    51073 non-null  float64
 8   捷運出口_under_750_m    51073 non-null  float64
 9   宗教場所_under_250_m    51073 non-null  float64
 10  宗教場所_under_500_m    51073 non-null  float64
 11  宗教場所_under_750_m    51073 non-null  float64
 12  消防單位_under_250_m    51073 non-null  float64
 13  消防單位_under_500_m    51073 non-null  float64
 14  消防單位_under_750_m    51073 non-null  float64
 15  商圈_under_250_m      51073 non-null  float64
 16  商圈_u

In [16]:
new_col = [
    'Public_Activity_Space_under_250_m', 'Public_Activity_Space_under_500_m', 'Public_Activity_Space_under_750_m',
    'Interchange_Entrance_under_250_m', 'Interchange_Entrance_under_500_m', 'Interchange_Entrance_under_750_m',
    'MRT_Exit_under_250_m', 'MRT_Exit_under_500_m', 'MRT_Exit_under_750_m',
    'Religious_Site_under_250_m', 'Religious_Site_under_500_m', 'Religious_Site_under_750_m',
    'Fire_Department_under_250_m', 'Fire_Department_under_500_m', 'Fire_Department_under_750_m',
    'Shopping_Area_under_250_m', 'Shopping_Area_under_500_m', 'Shopping_Area_under_750_m',
    'Library_under_250_m', 'Library_under_500_m', 'Library_under_750_m',
    'School_under_250_m', 'School_under_500_m', 'School_under_750_m',
    'Art_Culture_under_250_m', 'Art_Culture_under_500_m', 'Art_Culture_under_750_m',
    'Hospital_under_250_m', 'Hospital_under_500_m', 'Hospital_under_750_m',
    'Gas_Station_under_250_m', 'Gas_Station_under_500_m', 'Gas_Station_under_750_m',
    'Kindergarten_under_250_m', 'Kindergarten_under_500_m', 'Kindergarten_under_750_m',
    'Daycare_Center_under_250_m', 'Daycare_Center_under_500_m', 'Daycare_Center_under_750_m',
    'Court_Prosecutors_Office_under_250_m', 'Court_Prosecutors_Office_under_500_m', 'Court_Prosecutors_Office_under_750_m',
    'Convenience_Store_under_250_m', 'Convenience_Store_under_500_m', 'Convenience_Store_under_750_m',
    'Power_Plant_under_250_m', 'Power_Plant_under_500_m', 'Power_Plant_under_750_m',
    'Cleaning_Unit_under_250_m', 'Cleaning_Unit_under_500_m', 'Cleaning_Unit_under_750_m',
    'Incineration_Plant_under_250_m', 'Incineration_Plant_under_500_m', 'Incineration_Plant_under_750_m',
    'Bank_under_250_m', 'Bank_under_500_m', 'Bank_under_750_m',
    'Funeral_Home_under_250_m', 'Funeral_Home_under_500_m', 'Funeral_Home_under_750_m',
    'Police_Station_under_250_m', 'Police_Station_under_500_m', 'Police_Station_under_750_m',
    'Nursing_Home_under_250_m', 'Nursing_Home_under_500_m', 'Nursing_Home_under_750_m',
    'Clinic_under_250_m', 'Clinic_under_500_m', 'Clinic_under_750_m',
    'Pharmacy_under_250_m', 'Pharmacy_under_500_m', 'Pharmacy_under_750_m',
    'Medical_Facility_under_250_m', 'Medical_Facility_under_500_m', 'Medical_Facility_under_750_m'
]

translation_dict = {
    '公共活動空間': 'Public_Activity_Space',
    '交流道出入口': 'Interchange_Entrance',
    '捷運出口': 'MRT_Exit',
    '宗教場所': 'Religious_Site',
    '消防單位': 'Fire_Department',
    '商圈': 'Shopping_Area',
    '圖書館': 'Library',
    '學校': 'School',
    '藝文': 'Art_Culture',
    '醫院': 'Hospital',
    '加油站': 'Gas_Station',
    '幼兒園': 'Kindergarten',
    '托兒所': 'Daycare_Center',
    '法院_檢察署': 'Court_Prosecutors_Office',
    '便利商店': 'Convenience_Store',
    '發電廠': 'Power_Plant',
    '清潔單位': 'Cleaning_Unit',
    '焚化廠': 'Incineration_Plant',
    '銀行': 'Bank',
    '殯儀館': 'Funeral_Home',
    '警局': 'Police_Station',
    '療養院': 'Nursing_Home',
    '診所': 'Clinic',
    '藥局': 'Pharmacy',
    '醫療設施': 'Medical_Facility'
}

In [17]:
near_fac_df_2.columns = new_col
near_fac_df_2.columns

Index(['Public_Activity_Space_under_250_m',
       'Public_Activity_Space_under_500_m',
       'Public_Activity_Space_under_750_m', 'Interchange_Entrance_under_250_m',
       'Interchange_Entrance_under_500_m', 'Interchange_Entrance_under_750_m',
       'MRT_Exit_under_250_m', 'MRT_Exit_under_500_m', 'MRT_Exit_under_750_m',
       'Religious_Site_under_250_m', 'Religious_Site_under_500_m',
       'Religious_Site_under_750_m', 'Fire_Department_under_250_m',
       'Fire_Department_under_500_m', 'Fire_Department_under_750_m',
       'Shopping_Area_under_250_m', 'Shopping_Area_under_500_m',
       'Shopping_Area_under_750_m', 'Library_under_250_m',
       'Library_under_500_m', 'Library_under_750_m', 'School_under_250_m',
       'School_under_500_m', 'School_under_750_m', 'Art_Culture_under_250_m',
       'Art_Culture_under_500_m', 'Art_Culture_under_750_m',
       'Hospital_under_250_m', 'Hospital_under_500_m', 'Hospital_under_750_m',
       'Gas_Station_under_250_m', 'Gas_Station_under_

## 計算累加欄位

In [18]:
target_df_near_fac_df = near_fac_df_2.copy() # 並到新的表格比較不會出問題

for index in range(0,73,3): # 每個比鄰總類，切一個df
  tmp_each_fac_df = near_fac_df_2.iloc[:, index:index+3]
  fac_name = tmp_each_fac_df.columns[0].split('under')[0] # 取設施名稱
  target_df_near_fac_df[f'acc_{fac_name}under_500m'] = tmp_each_fac_df.iloc[:,0] + tmp_each_fac_df.iloc[:,1]
  target_df_near_fac_df[f'acc_{fac_name}under_750m'] = tmp_each_fac_df.iloc[:,0] + tmp_each_fac_df.iloc[:,1] + tmp_each_fac_df.iloc[:,2]


In [ ]:
target_df_near_fac_df.info(max_cols=125)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 125 columns):
 #    Column                                   Non-Null Count  Dtype  
---   ------                                   --------------  -----  
 0    Public_Activity_Space_under_250_m        51073 non-null  float64
 1    Public_Activity_Space_under_500_m        51073 non-null  float64
 2    Public_Activity_Space_under_750_m        51073 non-null  float64
 3    Interchange_Entrance_under_250_m         51073 non-null  float64
 4    Interchange_Entrance_under_500_m         51073 non-null  float64
 5    Interchange_Entrance_under_750_m         51073 non-null  float64
 6    MRT_Exit_under_250_m                     51073 non-null  float64
 7    MRT_Exit_under_500_m                     51073 non-null  float64
 8    MRT_Exit_under_750_m                     51073 non-null  float64
 9    Religious_Site_under_250_m               51073 non-null  float64
 10   Religious_Site_under_500_m      

In [19]:
# 把區間的欄位先丟掉
drop_list = []
for index in range(0,73,3): # 每個比鄰總類，切一個df
  tmp_each_fac_df = near_fac_df_2.iloc[:, index:index+3]
  drop_list.append(tmp_each_fac_df.columns[1])
  drop_list.append(tmp_each_fac_df.columns[2])

len(drop_list)

50

In [20]:
target_df_near_fac_df.drop(drop_list, axis=1, inplace=True)

In [ ]:
target_df_near_fac_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 75 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Public_Activity_Space_under_250_m        51073 non-null  float64
 1   Interchange_Entrance_under_250_m         51073 non-null  float64
 2   MRT_Exit_under_250_m                     51073 non-null  float64
 3   Religious_Site_under_250_m               51073 non-null  float64
 4   Fire_Department_under_250_m              51073 non-null  float64
 5   Shopping_Area_under_250_m                51073 non-null  float64
 6   Library_under_250_m                      51073 non-null  float64
 7   School_under_250_m                       51073 non-null  float64
 8   Art_Culture_under_250_m                  51073 non-null  float64
 9   Hospital_under_250_m                     51073 non-null  float64
 10  Gas_Station_under_250_m                  51073

## 低變異過濾(把主表丟掉的欄位也丟掉)

In [21]:
# 主表有使用到的欄位
need_columns = [
    "Public_Activity_Space_under_250_m",
    "MRT_Exit_under_250_m",
    "Religious_Site_under_250_m",
    "School_under_250_m",
    "Kindergarten_under_250_m",
    "Convenience_Store_under_250_m",
    "Bank_under_250_m",
    "Clinic_under_250_m",
    "Pharmacy_under_250_m",
    "acc_Public_Activity_Space_under_500m",
    "acc_Public_Activity_Space_under_750m",
    "acc_MRT_Exit_under_500m",
    "acc_MRT_Exit_under_750m",
    "acc_Religious_Site_under_500m",
    "acc_Religious_Site_under_750m",
    "acc_Shopping_Area_under_750m",
    "acc_School_under_500m",
    "acc_School_under_750m",
    "acc_Hospital_under_500m",
    "acc_Hospital_under_750m",
    "acc_Gas_Station_under_750m",
    "acc_Kindergarten_under_500m",
    "acc_Kindergarten_under_750m",
    "acc_Daycare_Center_under_500m",
    "acc_Daycare_Center_under_750m",
    "acc_Convenience_Store_under_500m",
    "acc_Convenience_Store_under_750m",
    "acc_Bank_under_500m",
    "acc_Bank_under_750m",
    "acc_Police_Station_under_750m",
    "acc_Clinic_under_500m",
    "acc_Clinic_under_750m",
    "acc_Pharmacy_under_500m",
    "acc_Pharmacy_under_750m",
    "acc_Medical_Facility_under_750m"
]

In [22]:
len(need_columns)

35

In [23]:
target_df_near_fac_df = target_df_near_fac_df.loc[:, need_columns]

## 把主表個的低變異欄位刪除

In [ ]:
near_fac_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 98 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           51073 non-null  int64  
 1   total_price          51073 non-null  int64  
 2   building_age         48588 non-null  float64
 3   total_area_ping      51073 non-null  float64
 4   address              51073 non-null  object 
 5   property_type        51073 non-null  object 
 6   usage_category       51073 non-null  object 
 7   latitude             51073 non-null  float64
 8   longitude            51073 non-null  float64
 9   community_name       51073 non-null  int64  
 10  has_parking_space    51073 non-null  int64  
 11  total_floors         51073 non-null  float64
 12  transaction_floors   51073 non-null  float64
 13  target_floors        51073 non-null  float64
 14  bedrooms             49395 non-null  float64
 15  living_rooms         48002 non-null 

In [24]:
nn_df = near_fac_df.copy()

In [ ]:
# dd_list = []
# for i in range(20,95):
#   dd_list.append(i)

In [ ]:
# nn_df.info(max_cols=155)
# nn_df.columns[dd_list]這樣的寫法其實就是把column name用出來，所以drop axis=1(col)就是要用column name
# nn_df.drop(nn_df.columns[dd_list], axis=1, inplace=True) # 若要用columns的index丟要這樣寫
# near_fac_df = df.iloc[:,31:106]

In [25]:
nn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 98 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           51073 non-null  int64  
 1   total_price          51073 non-null  int64  
 2   building_age         48588 non-null  float64
 3   total_area_ping      51073 non-null  float64
 4   address              51073 non-null  object 
 5   property_type        51073 non-null  object 
 6   usage_category       51073 non-null  object 
 7   latitude             51073 non-null  float64
 8   longitude            51073 non-null  float64
 9   community_name       51073 non-null  int64  
 10  has_parking_space    51073 non-null  int64  
 11  total_floors         51073 non-null  float64
 12  transaction_floors   51073 non-null  float64
 13  target_floors        51073 non-null  float64
 14  bedrooms             49395 non-null  float64
 15  living_rooms         48002 non-null 

In [26]:
final_df = pd.concat([nn_df,target_df_near_fac_df],axis=1)

In [27]:
final_df.info(max_cols=133)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 133 columns):
 #    Column                                Non-Null Count  Dtype  
---   ------                                --------------  -----  
 0    Unnamed: 0                            51073 non-null  int64  
 1    total_price                           51073 non-null  int64  
 2    building_age                          48588 non-null  float64
 3    total_area_ping                       51073 non-null  float64
 4    address                               51073 non-null  object 
 5    property_type                         51073 non-null  object 
 6    usage_category                        51073 non-null  object 
 7    latitude                              51073 non-null  float64
 8    longitude                             51073 non-null  float64
 9    community_name                        51073 non-null  int64  
 10   has_parking_space                     51073 non-null  int64  
 11   

In [ ]:
# final_df.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/c591_low_var.csv',encoding='utf-8',index=False)

In [ ]:
# final_df = pd.read_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/c591_low_var.csv',encoding='utf-8')

In [28]:
final_df['property_type'].isnull().sum()

0

# 基礎捕值fillna

## main_building_ratio
倉庫、工廠:、其他:、廠辦:、農舍:、辦公商業大樓:、透天厝:
這幾個幾乎全部都是nan
其中[倉庫, 工廠, 農舍, 辦公商業大樓, 透天厝]可以假設是"土地類的"，所以可以給100%
"其他"跟"廠辦"就比較難定義，那我覺得可以補全體的平均值

In [29]:
fillna_df = final_df.copy()

In [30]:
tmp = fillna_df[fillna_df['property_type']== '透天厝']

In [31]:
tmp['main_building_ratio'].describe()

count    3249.000000
mean       81.738643
std        15.131213
min         8.590000
25%        69.700000
50%        84.260000
75%        94.760000
max       100.000000
Name: main_building_ratio, dtype: float64

In [32]:
tmp['main_building_ratio'].isnull().sum()

724

In [33]:
fillna_df[fillna_df['property_type'] == '透天厝']['main_building_ratio'].info()

<class 'pandas.core.series.Series'>
Index: 3973 entries, 0 to 51055
Series name: main_building_ratio
Non-Null Count  Dtype  
--------------  -----  
3249 non-null   float64
dtypes: float64(1)
memory usage: 62.1 KB


In [34]:
type_p_list = fillna_df['property_type'].unique()

In [35]:
for type_p in type_p_list:
  tmp = fillna_df[fillna_df['property_type'] == type_p ]['main_building_ratio']
  need_fillna_index = tmp[ tmp.isnull() ].index

  # 錯誤示範: 簡單來說只要是切片索引下就不能就用fillna直接改變dataframe
  # fillna_df.loc[need_fillna_index, 'main_building_ratio'].fillna(tmp.mean(), inplace=True)

  # 正確反正我們已經找到特定property_type下na的index了，在原先的data_clean使用loc只需要在指定欄位main_building_ratio，我們就可以直接進行填值(mean)
  mean_value = tmp.mean()
  fillna_df.loc[need_fillna_index, 'main_building_ratio'] = mean_value

In [36]:
fillna_df['main_building_ratio'].isnull().sum()

0

## property_b_l_p_type

In [37]:
tt_df = fillna_df.copy()

In [38]:
tt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Columns: 133 entries, Unnamed: 0 to acc_Medical_Facility_under_750m
dtypes: float64(122), int64(5), object(6)
memory usage: 51.8+ MB


In [39]:
tt_df.drop(['Unnamed: 0'],axis=1, inplace=True)
# tt_df.drop(['Unnamed: 0', 'latitude', 'longitude', 'total_price', 'cor', 'address'],axis=1, inplace=True)

In [40]:
tt_df.info(max_cols=104)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Columns: 132 entries, total_price to acc_Medical_Facility_under_750m
dtypes: float64(122), int64(4), object(6)
memory usage: 51.4+ MB


## 要來填building age 和幾房廳

In [41]:
age_df = tt_df.copy()

In [42]:
age_list = age_df['property_type'].unique()

In [43]:
age_list = age_df['property_type'].unique()
for age_p in age_list:
  tmp = age_df[age_df['property_type'] == age_p ]['building_age']
  need_fillna_index = tmp[ tmp.isnull() ].index

  # 錯誤示範: 簡單來說只要是切片索引下就不能就用fillna直接改變dataframe
  # age_df.loc[need_fillna_index, 'main_building_ratio'].fillna(tmp.mean(), inplace=True)

  # 正確: 反正我們已經找到特定property_type下na的index了，在原先的data_clean使用loc只需要在指定欄位main_building_ratio，我們就可以直接進行填值(mean)
  mean_value = tmp.mean()
  age_df.loc[need_fillna_index, 'building_age'] = mean_value

In [44]:
age_df['building_age'].isnull().sum()

0

In [45]:
print('building_age')
for type_p in age_list:
  print(f"{type_p}:")
  tmp = age_df[age_df['property_type'] == type_p ]['building_age']
  print(f"{type_p} mean:{tmp.mean()}")
  print(f"{type_p}'s building_age null ratio is {tmp.isnull().sum()}/{len(tmp)}")
  print('-'*50)

building_age
透天厝:
透天厝 mean:31.961730192341626
透天厝's building_age null ratio is 0/3973
--------------------------------------------------
電梯大樓:
電梯大樓 mean:18.83881687753246
電梯大樓's building_age null ratio is 0/34145
--------------------------------------------------
公寓:
公寓 mean:42.187045587220716
公寓's building_age null ratio is 0/12955
--------------------------------------------------


In [ ]:
# age_df.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/EDA_fillna_building_age_ok.csv', encoding='utf-8', index=False) # '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/xgboost_month_freq_dataset.csv'

## 填幾房幾廳
* 這邊是一個一個填，沒有寫迴圈，要重做的話要注意
* 基本上都已property type為分類依據，去填值

In [46]:
many_room_df = age_df.copy()

In [47]:
many_room_df.info(max_cols=132)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 132 columns):
 #    Column                                Non-Null Count  Dtype  
---   ------                                --------------  -----  
 0    total_price                           51073 non-null  int64  
 1    building_age                          51073 non-null  float64
 2    total_area_ping                       51073 non-null  float64
 3    address                               51073 non-null  object 
 4    property_type                         51073 non-null  object 
 5    usage_category                        51073 non-null  object 
 6    latitude                              51073 non-null  float64
 7    longitude                             51073 non-null  float64
 8    community_name                        51073 non-null  int64  
 9    has_parking_space                     51073 non-null  int64  
 10   total_floors                          51073 non-null  float64
 11   

In [48]:
def fillna_with_house_rooms(col_name):
  house_set_list = many_room_df['property_type'].unique()
  for age_p in house_set_list:
    tmp = many_room_df[many_room_df['property_type'] == age_p ][col_name]
    need_fillna_index = tmp[ tmp.isnull() ].index

    # 錯誤示範: 簡單來說只要是切片索引下就不能就用fillna直接改變dataframe
    # many_room_df.loc[need_fillna_index, 'main_building_ratio'].fillna(tmp.mean(), inplace=True)

    # 正確反正我們已經找到特定property_type下na的index了，在原先的data_clean使用loc只需要在指定欄位main_building_ratio，我們就可以直接進行填值(mean)
    mean_value = tmp.mean().round(0)
    many_room_df.loc[need_fillna_index, col_name] = mean_value

  type_p_list = many_room_df['property_type'].unique()
  print(f"{col_name}")
  print('='*50)
  for type_p in type_p_list:
    print(f"{type_p}:")
    tmp = many_room_df[many_room_df['property_type'] == type_p ][col_name]
    print(f"{type_p} mean:{tmp.mean()}")
    print(f"{type_p}'s col_name null ratio is {tmp.isnull().sum()}/{len(tmp)}")
    print('-'*50)

In [49]:
rooms_list = many_room_df.columns[10:16]
for col_name in rooms_list:
  fillna_with_house_rooms(col_name)

total_floors
透天厝:
透天厝 mean:3.1313868613138687
透天厝's col_name null ratio is 0/3973
--------------------------------------------------
電梯大樓:
電梯大樓 mean:14.355835407819592
電梯大樓's col_name null ratio is 0/34145
--------------------------------------------------
公寓:
公寓 mean:4.814048629872636
公寓's col_name null ratio is 0/12955
--------------------------------------------------
transaction_floors
透天厝:
透天厝 mean:2.9219733199093882
透天厝's col_name null ratio is 0/3973
--------------------------------------------------
電梯大樓:
電梯大樓 mean:1.0219065749011569
電梯大樓's col_name null ratio is 0/34145
--------------------------------------------------
公寓:
公寓 mean:1.039675800849093
公寓's col_name null ratio is 0/12955
--------------------------------------------------
target_floors
透天厝:
透天厝 mean:0.9567077774981123
透天厝's col_name null ratio is 0/3973
--------------------------------------------------
電梯大樓:
電梯大樓 mean:7.857490115683116
電梯大樓's col_name null ratio is 0/34145
----------------------------------------

In [50]:
many_room_df.isnull().sum().sum()

0

In [51]:
type_p_list = many_room_df['property_type'].unique()
print('transaction_floors')
print('='*50)
for type_p in type_p_list:
  print(f"{type_p}:")
  tmp = many_room_df[many_room_df['property_type'] == type_p ]['transaction_floors']
  print(f"{type_p} mean:{tmp.mean()}")
  print(f"{type_p}'s transaction_floors null ratio is {tmp.isnull().sum()}/{len(tmp)}")
  print('-'*50)

transaction_floors
透天厝:
透天厝 mean:2.9219733199093882
透天厝's transaction_floors null ratio is 0/3973
--------------------------------------------------
電梯大樓:
電梯大樓 mean:1.0219065749011569
電梯大樓's transaction_floors null ratio is 0/34145
--------------------------------------------------
公寓:
公寓 mean:1.039675800849093
公寓's transaction_floors null ratio is 0/12955
--------------------------------------------------


In [52]:
house_set_list = many_room_df['property_type'].unique()
for age_p in house_set_list:
  tmp = many_room_df[many_room_df['property_type'] == age_p ]['transaction_floors']
  need_fillna_index = tmp[ tmp.isnull() ].index

  # 錯誤示範: 簡單來說只要是切片索引下就不能就用fillna直接改變dataframe
  # many_room_df.loc[need_fillna_index, 'main_building_ratio'].fillna(tmp.mean(), inplace=True)

  # 正確反正我們已經找到特定property_type下na的index了，在原先的data_clean使用loc只需要在指定欄位main_building_ratio，我們就可以直接進行填值(mean)
  mean_value = tmp.mean().round(0)
  many_room_df.loc[need_fillna_index, 'transaction_floors'] = mean_value

In [53]:
many_room_df.info(max_cols=104)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Columns: 132 entries, total_price to acc_Medical_Facility_under_750m
dtypes: float64(122), int64(4), object(6)
memory usage: 51.4+ MB


## usage category

In [54]:
many_room_df.isnull().sum().sum()

0

In [55]:
uu_df = many_room_df.copy()

In [56]:
uu_df['usage_category'].isnull().sum()

0

In [57]:
uu_df['usage_category'].unique()

array(['住家用', '集合住宅', '住商用', '住宅'], dtype=object)

In [58]:
# 把null值跟"6","7"，都改成"其他"
uu_df['usage_category'].fillna(value="其他", inplace=True)

for index, value in enumerate(uu_df['usage_category']):
    if value == "集合住宅":
        uu_df['usage_category'].iloc[index] = "住家用"
    if value == "住宅":
        uu_df['usage_category'].iloc[index] = "住家用"

串流輸出內容已截斷至最後 5000 行。
<ipython-input-58-b06dd71f1bcb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uu_df['usage_category'].iloc[index] = "住家用"
<ipython-input-58-b06dd71f1bcb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uu_df['usage_category'].iloc[index] = "住家用"
<ipython-input-58-b06dd71f1bcb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uu_df['usage_category'].iloc[index] = "住家用"
<ipython-input-58-b06dd71f1bcb>:6: SettingW

In [59]:
uu_df.isnull().sum().sum()

0

In [60]:
uu_df.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/c591_final_version_tmp_after_fillna.csv', encoding='utf-8', index=False) # '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/xgboost_month_freq_dataset.csv'

In [ ]:
uu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Columns: 132 entries, total_price to acc_Medical_Facility_under_750m
dtypes: float64(122), int64(4), object(6)
memory usage: 51.4+ MB


# 經濟指標(初挑)

In [ ]:
eco_df_origin = uu_df.copy()

In [ ]:
eco_df_origin.info(max_cols=104)

In [ ]:
eco_df_eco = eco_df_origin.iloc[:,24:65]

In [ ]:
eco_df_eco.columns

In [ ]:
eco_drop_list = ['EA1101', 'EB0312', 'EB0911',   'EB22', 'EC06', 'EC07',
       'LA07', 'MA1501', 'MB65',    'MC36G', 'MO30',
       'MP101G',
       'NE0812', 'OC08G',
         '倫敦鋁現貨價', '倫敦高級銅極現貨價',
       '巴拿馬型貨櫃運費指數', '杜拜輕原油現貨價', '水泥50kg北區台泥', '無鉛汽油95', '竹節鋼筋_南',
       ]

In [ ]:
eco_df_origin.drop(eco_drop_list, axis=1, inplace=True)

In [ ]:
eco_file_path = '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/house_index.csv'
eco_df = pd.read_csv(eco_file_path)

In [ ]:
eco_df['date'] = pd.to_datetime(eco_df['date'])
eco_df['year_month_for_combine'] = eco_df['date'].dt.to_period('M')

In [ ]:
eco_df.info()

In [ ]:
eco_df_origin['date'] = pd.to_datetime(eco_df_origin['date'])
eco_df_origin['year_month_for_combine'] = eco_df_origin['date'].dt.to_period('M')

In [ ]:
final_dff = eco_df_origin.merge(eco_df, left_on='year_month_for_combine', right_on='year_month_for_combine', how='inner')

In [ ]:
final_dff.info()

In [ ]:
final_dff.drop(['date_y', 'year', 'month', 'transaction_date', 'date_x'],axis=1, inplace=True)

In [ ]:
final_dff.info()

In [ ]:
another_df = final_dff[final_dff['per_ping_calculation_method'] != '總面積-車位總面積 剩餘面積小於1平方公尺 ,不計算單價']

In [ ]:
another_df.drop('per_ping_calculation_method',axis=1,inplace=True)

In [ ]:
another_df.drop('address',axis=1,inplace=True)

In [ ]:
another_df['property_b_l_p_type']

# 編碼
* 最好先處理下面的usage

In [61]:
another_df = uu_df.copy()

In [62]:
another_df['property_type_on'] =another_df['property_type']
another_df['city_on'] = another_df['city']
another_df['district_on'] = another_df['district']
another_df['usage_category_on'] = another_df['usage_category']

In [64]:
another_df.info(max_cols=136)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 136 columns):
 #    Column                                Non-Null Count  Dtype  
---   ------                                --------------  -----  
 0    total_price                           51073 non-null  int64  
 1    building_age                          51073 non-null  float64
 2    total_area_ping                       51073 non-null  float64
 3    address                               51073 non-null  object 
 4    property_type                         51073 non-null  object 
 5    usage_category                        51073 non-null  object 
 6    latitude                              51073 non-null  float64
 7    longitude                             51073 non-null  float64
 8    community_name                        51073 non-null  int64  
 9    has_parking_space                     51073 non-null  int64  
 10   total_floors                          51073 non-null  float64
 11   

In [65]:
# 會自己把原先欄位drop掉
another_df = pd.get_dummies(another_df, columns=['property_type_on'], prefix='property_type_')
another_df = pd.get_dummies(another_df, columns=['city_on'], prefix='city_')
another_df = pd.get_dummies(another_df, columns=['district_on'], prefix='district_')
another_df = pd.get_dummies(another_df, columns=['usage_category_on'], prefix='usage_category_')

In [67]:
another_df.info(max_cols=177)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51073 entries, 0 to 51072
Data columns (total 177 columns):
 #    Column                                Non-Null Count  Dtype  
---   ------                                --------------  -----  
 0    total_price                           51073 non-null  int64  
 1    building_age                          51073 non-null  float64
 2    total_area_ping                       51073 non-null  float64
 3    address                               51073 non-null  object 
 4    property_type                         51073 non-null  object 
 5    usage_category                        51073 non-null  object 
 6    latitude                              51073 non-null  float64
 7    longitude                             51073 non-null  float64
 8    community_name                        51073 non-null  int64  
 9    has_parking_space                     51073 non-null  int64  
 10   total_floors                          51073 non-null  float64
 11   

In [68]:
another_df.iloc[:,132:] = another_df.iloc[:,132:].astype(int)

In [69]:
another_df.isnull().sum().sum()

0

In [70]:
another_df['has_parking_space'].value_counts()

has_parking_space
0    26551
1    24522
Name: count, dtype: int64

In [72]:
another_df.isnull().sum().sum()

0

In [73]:
another_df.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/c591_final_versoin_0711.csv', encoding='utf-8', index=False) # '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/xgboost_month_freq_dataset.csv'

In [74]:
another_df['district'].unique()

array(['南港區', '松山區', '內湖區', '文山區', '信義區', '中山區', '大安區', '北投區', '中正區',
       '士林區', '萬華區', '大同區', '中和區', '新莊區', '淡水區', '板橋區', '三重區', '新店區',
       '泰山區', '林口區', '土城區', '汐止區', '三芝區', '樹林區', '五股區', '蘆洲區', '八里區',
       '永和區', '深坑區', '鶯歌區', '三峽區', '萬里區', '金山區', '瑞芳區', '坪林區', '貢寮區',
       '烏來區', '石門區'], dtype=object)

# 補看一下usage，591應該沒用這一塊

In [ ]:
usage_df = final_dff.copy()

In [ ]:
final_dff.info()

In [ ]:
usage_df['usage_category'].value_counts()

In [ ]:
ee_df1 = usage_df[ (usage_df['usage_category'] != '工')]

In [ ]:
eee_df2 = ee_df1[usage_df['usage_category'] != '農']

In [ ]:
eee_df2['usage_category'].isnull().sum().sum()

In [ ]:
eee_df2 = eee_df2[eee_df2['per_ping_calculation_method'] != '總面積-車位總面積 剩餘面積小於1平方公尺 ,不計算單價']

In [ ]:
eee_df2.drop('per_ping_calculation_method',axis=1,inplace=True)

In [ ]:
eee_df2.drop('address',axis=1,inplace=True)

In [ ]:
# 會自己把原先欄位drop掉
eee_df2 = pd.get_dummies(eee_df2, columns=['property_type'], prefix='property_type_')
eee_df2 = pd.get_dummies(eee_df2, columns=['city'], prefix='city_')
eee_df2 = pd.get_dummies(eee_df2, columns=['property_b_l_p_type'], prefix='property_b_l_p_type_')
eee_df2 = pd.get_dummies(eee_df2, columns=['district'], prefix='district_')
eee_df2 = pd.get_dummies(eee_df2, columns=['usage_category'], prefix='usage_category_')

In [ ]:
eee_df2.info()

In [ ]:
eee_df2['property_b_l_p_type__0']

In [ ]:
eee_df2.drop(['property_b_l_p_type__0'],axis=1,inplace=True)

In [ ]:
eee_df2.info(max_cols=129)

In [ ]:
eee_df2.iloc[:,75:129] = eee_df2.iloc[:,75:129].astype(int)

In [ ]:
eee_df2.info(max_cols=129)

In [ ]:
# eee_df2.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/EDA_main_data_for_xgb_version4_usage_fix.csv', encoding='utf-8', index=False) # '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/xgboost_month_freq_dataset.csv'